In [2]:
import os
import glob
import pandas as pd
import numpy as np

# 폴더 경로 지정
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본"

# 해당 폴더 내 모든 CSV 파일 가져오기
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file_path in csv_files:
    try:
        # 파일명에서 공백이 있으면 _로 대체
        dir_name, file_name = os.path.split(file_path)
        new_file_name = file_name.replace(' ', '_')
        new_file_path = os.path.join(dir_name, new_file_name)

        if file_name != new_file_name:
            os.rename(file_path, new_file_path)
            print(f"Renamed file: {file_name} -> {new_file_name}")
            file_path = new_file_path  # 경로 업데이트

        # ===== 안정적 읽기 옵션 (핵심 수정) =====
        # - engine='python': 다양한 구분자/불일치 라인 허용
        # - sep=r'[,\t;]+' : 콤마/탭/세미콜론 혼합 구분자 허용
        # - on_bad_lines='skip': 깨진 라인(따옴표 미매칭 등) 건너뛰기
        read_opts_preview = dict(
            header=None, nrows=30, engine='python',
            sep=r'[,\t;]+', on_bad_lines='skip',
            encoding='utf-8-sig'
        )
        read_opts_full = dict(
            header=None, skiprows=21, engine='python',
            sep=r'[,\t;]+', on_bad_lines='skip',
            encoding='utf-8-sig'
        )

        # ---- 안전장치: 처음 30줄 확인 (원본 로직 유지) ----
        preview = pd.read_csv(file_path, **read_opts_preview)
        contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
        if not contains_text:
            print(f"Skipped (이미 처리된 파일입니다.): {file_path}")
            continue

        # 21행 스킵 (열 개수와 무관) - 유연 구분자/불량라인 스킵 적용
        df = pd.read_csv(file_path, **read_opts_full)

        # 숫자 변환 + NaN-only 행 제거 (원본 로직 유지)
        df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all')
        if df.empty:
            print(f"No valid data in {file_path}")
            continue

        # NumPy 변환
        data = df.values
        num_rows, num_cols = data.shape

        # ----- 시간 재계산 (원본 기능 유지: 첫 열만 등간격으로 교체) -----
        if num_rows > 1 and num_cols >= 1:
            start_time = data[0, 0]
            end_time   = data[-1, 0]

            # 시작/끝이 NaN일 경우 기본 간격 1.0으로 대체
            if pd.isna(start_time) or pd.isna(end_time):
                time_interval = 1.0
            else:
                time_interval = (end_time - start_time) / (num_rows - 1)

            time_col = np.arange(num_rows) * time_interval
            data[:, 0] = time_col  # 첫 번째 열만 새 시간으로 교체

        # 파일 저장 (덮어쓰기) - 열 개수 유지
        np.savetxt(file_path, data, delimiter=',', fmt='%.10f')
        print(f"Filtered data saved at: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")


Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본\Aloe_DEG_1.csv
Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본\Aloe_DEG_2.csv
Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본\Aloe_DEG_3.csv
Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본\Aloe_DEG_4.csv
Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251019_데모 데이터\csv파일 - 복사본\conv2-1.csv
Skipped (이미 처리된 파일입니다.): C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data 

C:\Users\user\AppData\Local\Temp\ipykernel_22816\853471588.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_22816\853471588.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_22816\853471588.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_22816\853471588.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_22816\853471588.py:41: FutureWarning: DataFrame.applymap 